In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import sys, os, time, collections

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'cells'))
# sys.path.append(os.path.join(os.environ['REPO_DIR'], 'annotation'))
# sys.path.append(os.path.join(os.environ['REPO_DIR']bb, 'learning'))

import numpy as np
import scipy
import skimage
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from skimage.measure import regionprops, label, find_contours
from skimage.color import rgb2hsv
from scipy import ndimage as ndi
import pandas
from scipy.signal import argrelmax
from scipy.stats import linregress

from annotation_utilities import *
from registration_utilities import *
from learning_utilities import *
from data_manager import *
from utilities2015 import *
from cell_utilities import *
from metadata import *
from distributed_utilities import download_from_s3

save_folder_path = '/shared/MouseBrainAtlasXiang/XJ/Output/Collect_typical_blobs/'
from xj_utilities import *
fun_create_folder(save_folder_path=save_folder_path)

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
No vtk


Setting environment for AWS compute node


Cannot import mxnet.
Folder already exists: /shared/MouseBrainAtlasXiang/XJ/Output/Collect_typical_blobs/

In [3]:
def fun_get_regionprop(skimage_measure_region, prop):
    """skimage_measure_region: regionprops, returned by skimage.measure.regionprps(), of the type skimage.measure.regionprops
        prop: region property, can be built-in properties like 'area', 'moments_hu', eccentricity', or in addition, 'compactness'
    Returns: regionprops[prop] in float/numpy.ndarray/int, depends on the input.  """
    buildinProps = ['coords','centroid','orientation', 'eccentricity','area','orientation','moments_hu','bbox','equivalent_diameter','label','local_centroid','major_axis_length','solidity','minor_axis_length','perimeter','solidity']
    PI = 3.1415926
    if prop in buildinProps:
        return skimage_measure_region[prop]
    elif prop == 'compactness':
        return float(skimage_measure_region['perimeter'])**2 / float(skimage_measure_region['area']) / (4 * PI)

def fun_get_regionprop_list(skimage_measure_region_list, prop):
    regionprop_list = [fun_get_regionprop(record, prop) for record in skimage_measure_region_list]
    return np.array(regionprop_list)

def fun_save_typical_blobs_info(typical_blobs, prop_to_save, stack, section, dataType='typical'):
    # Save tpical blob information
    typical_blob_regionprops = [record[2] for record in typical_blobs]
    fun_save_regionprops(regionprop_List=typical_blob_regionprops,prop_to_save=prop_to_save,stack=stack,sec=section,dataType=dataType)

def fun_get_matched_blobs_IDs(matched_paris):
    ''' matched_pairs: a list of [section, blobID, typical_blob_props, scan_sec, matched_blob, matched_blob_props, sim_matrix]
        return: 
            matched_blobs_from_section: a list of sections indicates where do the matched blobs come from
            matched_blobs_with_typicalBlob: a list of typical_blob_ID indicates which typical blob is matched'''
    matched_blobs_from_section = []
    matched_blobs_with_typicalBlob = []
    typical_blobs_id = -1
    blobID = -1
    for record in matched_paris:
        if blobID != record[1]:
            blobID = record[1]
            typical_blobs_id +=1
        temp_matched_blobs_section = record[3]
        temp_num_matched_blobs = len(record[4])
        matched_blobs_from_section += [temp_matched_blobs_section] * temp_num_matched_blobs
        matched_blobs_with_typicalBlob += [typical_blobs_id] * temp_num_matched_blobs
    return matched_blobs_from_section, matched_blobs_with_typicalBlob    
    
    
def fun_save_matched_blobs_info(matched_paris, prop_to_save, stack, section, dataType='matched'):
    matched_blob_regionprops = [record[5] for record in matched_paris]
    matched_blob_regionprops = np.concatenate(tuple(matched_blob_regionprops)).tolist()
    fun_save_regionprops(regionprop_List=matched_blob_regionprops, prop_to_save=prop_to_save, stack=stack, sec=section, dataType=dataType)
    
    matched_blobs_from_section, matched_blobs_with_typicalBlob = fun_get_matched_blobs_IDs(matched_paris=matched_paris)
    matched_blobs_from_section = np.array(matched_blobs_from_section, dtype=np.int16)
    matched_blobs_with_typicalBlob = np.array(matched_blobs_with_typicalBlob, dtype=np.int16)
    bp.pack_ndarray_file(matched_blobs_with_typicalBlob, get_typical_cell_data_filepath(what='TBID', stack=stack, sec=section, dataType=dataType) )
    bp.pack_ndarray_file(matched_blobs_from_section, get_typical_cell_data_filepath(what='section', stack=stack, sec=section, dataType=dataType) )
   

In [4]:
def fun_save_regionprops(regionprop_List, prop_to_save, stack, sec, dataType='typical'):
    for tempProp in prop_to_save:
        tempProp_data = fun_get_regionprop_list(skimage_measure_region_list=regionprop_List, prop=tempProp)
        
        if tempProp == 'coords':
            tempProp_data = map(lambda data: np.array(data, dtype=np.int16), tempProp_data)
        elif tempProp == 'moments_hu':
            tempProp_data = map(lambda data: np.array(data, dtype=np.float32), tempProp_data)
            tempProp_data = np.row_stack(tuple(tempProp_data))
        elif tempProp == 'centroid':
            tempProp_data = map(lambda data: np.array(data, dtype=np.float32), tempProp_data)
            tempProp_data = np.row_stack(tuple(tempProp_data))
        elif tempProp == 'area':
            tempProp_data = np.array(tempProp_data, np.int32)
        elif tempProp == 'eccentricity':
            tempProp_data = np.array(tempProp_data, np.float32)
        elif tempProp == 'equivalent_diameter':
            tempProp_data = np.array(tempProp_data, np.float32)
        elif tempProp == 'compactness':
            tempProp_data = np.array(tempProp_data, np.float32)
        elif tempProp == 'perimeter':
             tempProp_data = np.array(tempProp_data, np.int32)

        tempFp = get_typical_cell_data_filepath(what=tempProp,stack=stack,sec=sec, dataType=dataType)
        create_if_not_exists(os.path.dirname(tempFp))
        if tempFp.endswith('.hdf'):
            save_hdf_v2(tempProp_data, fn=tempFp)
        elif tempFp.endswith('.bp'):
            bp.pack_ndarray_file(tempProp_data, tempFp)
        else:
            print 'Unrecognized data type. Save failed.'
    return 0

In [5]:
scan_parameters = {}

In [6]:
scan_parameters['stack'] = 'MD589'
scan_parameters['patch_size'] = 448
scan_parameters['patch_half_size'] = scan_parameters['patch_size']/2
scan_parameters['stride'] = 112

scan_parameters['section_limits'] = metadata_cache['section_limits'][scan_parameters['stack']]
scan_parameters['o_crop'] = True
scan_parameters['o_clear_border'] = True
scan_parameters['o_relabel'] = True
scan_parameters['oriImL1'], scan_parameters['oriImL0'] = metadata_cache['image_shape'][scan_parameters['stack']]
scan_parameters['scan_section_range'] = 1

scan_parameters['prop'] = ['centroid','eccentricity','area','orientation','moments_hu','bbox','equivalent_diameter','perimeter','compactness','label','major_axis_length','minor_axis_length']
scan_parameters['prop_for_comparison'] = ['area', 'eccentricity']
if 'moments_hu' in scan_parameters['prop_for_comparison']:
    scan_parameters['compare_weight'] = [1 for i in range(6 + len(scan_parameters['prop_for_comparison']))]
else:
    scan_parameters['compare_weight'] = [1 for i in range(len(scan_parameters['prop_for_comparison']))]
scan_parameters['compare_weight'] = np.array(scan_parameters['compare_weight'])/ float(np.sum(scan_parameters['compare_weight']))
scan_parameters['similarity_threshold'] = 0.9
scan_parameters['o_fix_scan_size'] = True
scan_parameters['scan_size'] = 112
scan_parameters['scan_size_coeff'] = 5
scan_parameters['builtInProps'] = ['centroid','orientation', 'eccentricity','area','orientation','moments_hu','bbox','equivalent_diameter','label','local_centroid','major_axis_length','solidity','minor_axis_length','perimeter','solidity']
scan_parameters['prop_to_save'] = ['coords','moments_hu','centroid','area','eccentricity','equivalent_diameter', 'compactness']
scan_parameters['scan_outside_margin'] = 1000
stack = scan_parameters['stack']

In [4]:
contour_df = DataManager.load_annotation_v4(stack=scan_parameters['stack'],by_human=True,suffix='contours',timestamp='latest')
contour_df = convert_annotation_v3_original_to_aligned_cropped(contour_df=contour_df,stack=stack)
structure_name = '7N'
vertice_7N_dic = {record['section']:np.array(record['vertices'],np.int) for _, record in contour_df[contour_df['name']==structure_name].iterrows()}
sectionList_7N = np.sort(vertice_7N_dic.keys())

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD589" "/shared/CSHL_labelings_v3/MD589" --exclude "*" --include "*contours*"
latest timestamp:  07292017045157


In [11]:
# Whole image setting:
scan_parameters['crop_0_min'] = 0
scan_parameters['crop_1_min'] = 0
scan_parameters['crop_0_max'] = scan_parameters['oriImL0']
scan_parameters['crop_1_max'] = scan_parameters['oriImL1']
scan_parameters['crop_range_mmxx'] = (scan_parameters['crop_0_min'], scan_parameters['crop_1_min'], scan_parameters['crop_0_max'], scan_parameters['crop_1_max'])
scan_parameters['crop_range_mxmx'] = fun_mmxx_to_mxmx(*scan_parameters['crop_range_mmxx'])
scan_parameters['im0max'] = scan_parameters['crop_0_max'] - scan_parameters['crop_0_min']
scan_parameters['im1max'] = scan_parameters['crop_1_max'] - scan_parameters['crop_1_min']

In [5]:
# margin = 10
# bboxs_7N = {sec:fun_polygon_bbox(vertice_7N_dic[sec]) for sec in sectionList_7N}
# bbox_7N = fun_polygons_bbox(bboxs_7N.values(),margin=margin)
# # bbox_7N = bboxs_7N[154]
# scan_parameters['crop_range_mmxx'] = bbox_7N
# scan_parameters['crop_0_min'], scan_parameters['crop_1_min'],scan_parameters['crop_0_max'],scan_parameters['crop_1_max'] = bbox_7N 
# scan_parameters['im0max'] = scan_parameters['crop_0_max'] - scan_parameters['crop_0_min']
# scan_parameters['im1max'] = scan_parameters['crop_1_max'] - scan_parameters['crop_1_min']
# scan_parameters['crop_range_mxmx'] = fun_mmxx_to_mxmx(*scan_parameters['crop_range_mmxx'])

In [8]:
# Get avaliable sections:
stack = scan_parameters['stack']
section_begin, section_end = scan_parameters['section_limits']
load_section_num = 100
load_start_section = 152 - section_begin
all_valid_section = fun_get_valid_section_list(stack=stack)
# secList = all_valid_section[load_start_section:load_section_num + load_start_section]
# secList = list(set(all_valid_section).intersection(sectionList_7N))
secList = all_valid_section
scan_parameters['secList'] = secList

In [139]:
# # loading data
# image = {}
# for tempSec in secList:
#     try:
#         image[tempSec] = fun_crop_images(DataManager.load_image_v2(stack=stack, section=tempSec, version='jpeg', prep_id=2),*scan_parameters['crop_range_mmxx'],im0max=scan_parameters['oriImL0'],im1max=scan_parameters['oriImL1'] )
#     except:
#         sys.stderr.write('Cannot import section %d...\n'%tempSec)
#         continue

In [18]:
def fun_collect_blobs(section, scan_parameters, collectFlaseMatched=False, saveMatched=True):
    scan_range = scan_parameters['scan_section_range']
    im0max = scan_parameters['im0max']
    im1max = scan_parameters['im1max']
    prop = scan_parameters['prop']
    prop_for_comparison = scan_parameters['prop_for_comparison']
    compare_weight = scan_parameters['compare_weight']
    o_simil_threshold = scan_parameters['similarity_threshold']
    o_fix_scan_size = scan_parameters['o_fix_scan_size']
    o_scan_size_coeff = scan_parameters['scan_size_coeff']
    o_scan_size = scan_parameters['scan_size']  
    secList = scan_parameters['secList'] 
    stack = scan_parameters['stack']

    # data_typical_blobs = {}
    # data_matched_paris = {}
    # data_false_typical_blobs = {}
    # data_false_matched_pairs = {}
    # data_blob_prop_dic = {}
    # data_blob_idx_selection = {}
#     section = 154
    # for section in secList:
    typical_blobs = []
    matched_paris = []
    false_typical_blobs = []
    false_matched_paris = []
    cell_centroids = {}
    cell_numbers = {}
    cell_global_coord = {}
    im_blob_prop = {}
    im_label = {}
    im_BW = {}
    sec_load_data_list = range(section - scan_range, section + scan_range + 1)
    scan_section = list(sec_load_data_list)
    scan_section.remove(section)
    ### loading data and reconstructed labeled images ###
    tempList = list(sec_load_data_list)
    for tempSec in tempList:
        if tempSec in secList:
            cell_global_coord[tempSec] = load_cell_data('coords', stack=stack, sec=tempSec)
            _, temp_im_blob_prop, _ = fun_reconstruct_labeled_image(cell_global_coord[tempSec],crop_range= scan_parameters['crop_range_mxmx'], 
                                                                        oriImL0=scan_parameters['oriImL0'],oriImL1=scan_parameters['oriImL1'])
#             im_label[tempSec] = temp_im_label
#             im_BW[tempSec] = temp_im_label > 0
            im_blob_prop[tempSec] = np.array(temp_im_blob_prop)
        else:
            sys.stderr.write('Warning: missing section %d'%tempSec)
            scan_section.remove(tempSec)
            sec_load_data_list.remove(tempSec)

    ### Start getting region properties of each blob    ###
    blob_prop_dic = fun_regionprops_dic(im_blob_prop=im_blob_prop,scan_parameters=scan_parameters)
    # data_blob_prop_dic[section] = blob_prop_dic[section] # Save data
    blob_idx_selection = {tempSec : np.logical_and.reduce(np.row_stack(
        (blob_prop_dic[tempSec]['compactness']>0,
    #      blob_prop_dic[tempSec]['compactness']<2.5,
         blob_prop_dic[tempSec]['area']<3000,
         blob_prop_dic[tempSec]['area']>200))) for tempSec in sec_load_data_list}  

    # data_blob_idx_selection[section] = blob_idx_selection[section] # Save data

    n_blobs = {tempSec: len(im_blob_prop[tempSec]) for tempSec in im_blob_prop.keys()}
    secList_in_BlobPropDic = im_blob_prop.keys()
    if set(scan_section).issubset(set(secList_in_BlobPropDic)):
        pass
    else:
        print('Warrning: Scaned section(s) not included in input im_blob_prop')



    ### Start scanning ###
    for blobID in range(n_blobs[section]):
        if (blobID % 1000 == 0):
            print('Section %d Finished percentage: %f'%(section, (float(blobID)*100 / n_blobs[section]) ))

        temp_curr_blob_props = {}
        for tempProp in prop:
            temp_curr_blob_props[tempProp] = blob_prop_dic[section][tempProp][blobID]
        if not blob_idx_selection[section][blobID]:
    #         print('Blob %d eliminated'%blobID)
            continue

        tempB1_idx_loc = temp_curr_blob_props['centroid']
        if o_fix_scan_size:
            temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max,o_form='2D')
            temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max)
        else:
            temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max,o_form='2D')
            temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max)


        for tempSec in scan_section:
    #         print('Blbo %d left. Start scanning'%blobID)
            if tempSec not in secList_in_BlobPropDic:
                continue

            # Find blobs at the nearby location in the scaned section
            tempBlobInside = fun_blobs_in_polygen(blob_prop_dic[tempSec]['centroid'],temp_next_sec_range,coor_order='cr')
            tempBlobInsideIndex = np.where(tempBlobInside)[0]
            temp_num_blob = len(tempBlobInsideIndex)
            if collectFlaseMatched == True:
                tempBlobOutside = fun_blobs_out_polygen(blob_prop_dic[tempSec]['centroid'], temp_next_sec_range, coor_order='cr', margin=scan_parameters['scan_outside_margin'])
                tempBlobOutsideIndex = np.where(tempBlobOutside)[0]
                temp_num_outblob = len(tempBlobOutsideIndex)

            ### Typical blobs###

            if temp_num_blob:
                temp_sim = {}
                for temp_prop in prop_for_comparison:
                          temp_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobInside],distance_type=temp_prop))
                temp_sim_matrix = np.column_stack((temp_sim[temp_prop] for temp_prop in prop_for_comparison))

                #### Blob comparison ####
                temp_weighted_sim = np.dot(temp_sim_matrix,compare_weight)
                temp_compare_result = temp_weighted_sim > o_simil_threshold
                if any(temp_compare_result.tolist()):
                    typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                    matched_paris.append([section,blobID,
                                          im_blob_prop[section][blobID],
                                          tempSec,
                                          tempBlobInsideIndex[temp_compare_result],
                                          im_blob_prop[tempSec][tempBlobInsideIndex[temp_compare_result]],
                                          temp_sim_matrix[temp_compare_result,:]])
            else:
                pass

            ### False-typical blobs###
            if collectFlaseMatched == True:
                if temp_num_outblob:
                    temp_false_sim = {}
                    for temp_prop in prop_for_comparison:
                              temp_false_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobOutside],distance_type=temp_prop))
                    temp_false_sim_matrix = np.column_stack((temp_false_sim[temp_prop] for temp_prop in prop_for_comparison))

                    #### Blob comparison ####
                    temp_weighted_false_sim = np.dot(temp_false_sim_matrix,compare_weight)
                    temp_false_compare_result = temp_weighted_false_sim > o_simil_threshold
                    if any(temp_false_compare_result.tolist()):
                        false_typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                        false_matched_paris.append([section,blobID,im_blob_prop[section][blobID],
                                                    tempSec, 
                                                    tempBlobOutsideIndex[temp_false_compare_result], 
                                                    im_blob_prop[tempSec][tempBlobOutsideIndex[temp_false_compare_result]],
                                                    temp_false_sim_matrix[temp_false_compare_result,:]])
                else:
                    pass
    tempFp = get_typical_cell_data_filepath(what='scan_parameters',stack=stack,sec=section)
    create_if_not_exists(os.path.dirname(tempFp))
    save_pickle(scan_parameters,tempFp)
    regionprop_List = [tempRecord[2] for tempRecord in typical_blobs]
    fun_save_regionprops(regionprop_List=regionprop_List, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section)

    fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='typical')
    if saveMatched == True:
        fun_save_matched_blobs_info(matched_paris=matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='matched')
    if collectFlaseMatched == True:
        fun_save_typical_blobs_info(typical_blobs=false_typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_typical')
        fun_save_matched_blobs_info(matched_paris=false_matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_matched')
    print('Section %d result saved.'% section)
    return 0
# data_typical_blobs[section] = typical_blobs
# data_matched_paris[section] = matched_paris
# data_false_typical_blobs[section] = false_typical_blobs
# data_false_matched_pairs[section] = false_matched_paris   

In [133]:
from multiprocess import Pool

In [ ]:
pool = Pool(NUM_CORES/2)
pool.map(lambda sec: fun_collect_blobs(section=sec, scan_parameters=scan_parameters, collectFlaseMatched=False, saveMatched=False), secList)

rm -rf "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_blobCoords.hdf" && mkdir -p "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048"
rm -rf "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC19-2015.07.30-22.32.05_MD589_2_0056/MD589-IHC19-2015.07.30-22.32.05_MD589_2_0056_blobCoords.hdf" && mkdir -p "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC19-2015.07.30-22.32.05_MD589_2_0056"
rm -rf "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_1_0052/MD589-N18-2015.07.30-17.11.24_MD589_1_0052_blobCoords.hdf" && mkdir -p "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_1_0052"
rm -rf "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N22-2015.07.30-17.27.26_MD589_3_0066/MD589-N22-2015.07.30-17.27.26_MD589_3_0066_blobCoords.hdf" && mkdir -p "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N22-2015.07.30-17.27

aws s3 cp "s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD589/MD589-N23-2015.07.30-17.31.29_MD589_1_0067/MD589-N23-2015.07.30-17.31.29_MD589_1_0067_blobCoords.hdf" "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N23-2015.07.30-17.31.29_MD589_1_0067/MD589-N23-2015.07.30-17.31.29_MD589_1_0067_blobCoords.hdf"
rm -rf "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC21-2015.07.30-22.40.00_MD589_2_0062/MD589-IHC21-2015.07.30-22.40.00_MD589_2_0062_blobCoords.hdf" && mkdir -p "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC21-2015.07.30-22.40.00_MD589_2_0062"
aws s3 cp "s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD589/MD589-IHC21-2015.07.30-22.40.00_MD589_2_0062/MD589-IHC21-2015.07.30-22.40.00_MD589_2_0062_blobCoords.hdf" "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC21-2015.07.30-22.40.00_MD589_2_0062/MD589-IHC21-2015.07.30-22.40.00_MD589_2_0062_blobCoords.hdf"
rm -rf "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N26-2015.07.30-17.44.37_MD589_1_0076

Section 129 Finished percentage: 0.000000
Section 92 Finished percentage: 95.842358
Section 129 Finished percentage: 1.817389
Section 92 Finished percentage: 97.759206
Section 120 Finished percentage: 0.000000
Section 129 Finished percentage: 3.634778
Section 120 Finished percentage: 1.972192
Section 92 Finished percentage: 99.676053
Section 120 Finished percentage: 3.944384
Section 129 Finished percentage: 5.452166
Section 120 Finished percentage: 5.916576
Section 129 Finished percentage: 7.269555
Section 120 Finished percentage: 7.888768
Section 129 Finished percentage: 9.086944
Section 120 Finished percentage: 9.860960
Section 120 Finished percentage: 11.833153
Section 129 Finished percentage: 10.904333
Section 120 Finished percentage: 13.805345
Section 129 Finished percentage: 12.721721
Section 120 Finished percentage: 15.777537
Section 120 Finished percentage: 17.749729
Section 129 Finished percentage: 14.539110
Section 120 Finished percentage: 19.721921
Section 129 Finished perce

Section 138 Finished percentage: 4.854526
Section 101 Finished percentage: 11.405977
Section 110 Finished percentage: 89.247532
Section 147 Finished percentage: 14.817229
Section 138 Finished percentage: 6.472701
Section 110 Finished percentage: 91.106855
Section 101 Finished percentage: 13.306973
Section 147 Finished percentage: 16.298952
Section 138 Finished percentage: 8.090877
Section 110 Finished percentage: 92.966179
Section 147 Finished percentage: 17.780675
Section 101 Finished percentage: 15.207969
Section 138 Finished percentage: 9.709052
Section 110 Finished percentage: 94.825502
Section 147 Finished percentage: 19.262398
Section 101 Finished percentage: 17.108965
Section 138 Finished percentage: 11.327227
Section 110 Finished percentage: 96.684826
Section 147 Finished percentage: 20.744121
Section 101 Finished percentage: 19.009961
Section 138 Finished percentage: 12.945403
Section 147 Finished percentage: 22.225844
Section 110 Finished percentage: 98.544150
Section 138 Fin

Section 93 Finished percentage: 15.716166
Section 93 Finished percentage: 17.680687
Section 93 Finished percentage: 19.645208
Section 93 Finished percentage: 21.609728
Section 121 Finished percentage: 0.000000
Section 93 Finished percentage: 23.574249
Section 121 Finished percentage: 1.973165
Section 93 Finished percentage: 25.538770
Section 121 Finished percentage: 3.946330
Section 93 Finished percentage: 27.503291
Section 121 Finished percentage: 5.919495
Section 121 Finished percentage: 7.892660
Section 93 Finished percentage: 29.467811
Section 121 Finished percentage: 9.865825
Section 93 Finished percentage: 31.432332
Section 121 Finished percentage: 11.838990
Section 93 Finished percentage: 33.396853
Section 121 Finished percentage: 13.812155
Section 93 Finished percentage: 35.361374
Section 121 Finished percentage: 15.785320
Section 93 Finished percentage: 37.325894
Section 121 Finished percentage: 17.758485
Section 121 Finished percentage: 19.731650
Section 93 Finished percentag

# For debug

In [107]:
scan_range = scan_parameters['scan_section_range']
im0max = scan_parameters['im0max']
im1max = scan_parameters['im1max']
prop = scan_parameters['prop']
prop_for_comparison = scan_parameters['prop_for_comparison']
compare_weight = scan_parameters['compare_weight']
o_simil_threshold = scan_parameters['similarity_threshold']
o_fix_scan_size = scan_parameters['o_fix_scan_size']
o_scan_size_coeff = scan_parameters['scan_size_coeff']
o_scan_size = scan_parameters['scan_size']  
secList = scan_parameters['secList'] 
stack = scan_parameters['stack']

# data_typical_blobs = {}
# data_matched_paris = {}
# data_false_typical_blobs = {}
# data_false_matched_pairs = {}
# data_blob_prop_dic = {}
# data_blob_idx_selection = {}
section = 154
# for section in secList:
typical_blobs = []
matched_paris = []
false_typical_blobs = []
false_matched_paris = []
cell_centroids = {}
cell_numbers = {}
cell_global_coord = {}
im_blob_prop = {}
im_label_ori = {}
im_label = {}
im_BW = {}
sec_load_data_list = range(section - scan_range, section + scan_range + 1)
scan_section = list(sec_load_data_list)
scan_section.remove(section)
### loading data and reconstructed labeled images ###
tempList = list(sec_load_data_list)
for tempSec in tempList:
    if tempSec in secList:
        cell_global_coord[tempSec] = load_cell_data('coords', stack=stack, sec=tempSec)
        temp_im_label, temp_im_blob_prop, _ = fun_reconstruct_labeled_image(cell_global_coord[tempSec],crop_range= scan_parameters['crop_range_mxmx'], 
                                                                    oriImL0=scan_parameters['oriImL0'],oriImL1=scan_parameters['oriImL1'])
        im_label[tempSec] = temp_im_label
        im_BW[tempSec] = temp_im_label > 0
        im_blob_prop[tempSec] = np.array(temp_im_blob_prop)
    else:
        sys.stderr.write('Warning: missing section %d'%tempSec)
        scan_section.remove(tempSec)
        sec_load_data_list.remove(tempSec)

### Start getting region properties of each blob    ###
blob_prop_dic = fun_regionprops_dic(im_blob_prop=im_blob_prop,scan_parameters=scan_parameters)
# data_blob_prop_dic[section] = blob_prop_dic[section] # Save data
blob_idx_selection = {tempSec : np.logical_and.reduce(np.row_stack(
    (blob_prop_dic[tempSec]['compactness']>0,
#      blob_prop_dic[tempSec]['compactness']<2.5,
     blob_prop_dic[tempSec]['area']<3000,
     blob_prop_dic[tempSec]['area']>200))) for tempSec in sec_load_data_list}  

# data_blob_idx_selection[section] = blob_idx_selection[section] # Save data

n_blobs = {tempSec: len(im_blob_prop[tempSec]) for tempSec in im_blob_prop.keys()}
secList_in_BlobPropDic = im_blob_prop.keys()
if set(scan_section).issubset(set(secList_in_BlobPropDic)):
    pass
else:
    print('Warrning: Scaned section(s) not included in input im_blob_prop')



### Start scanning ###
for blobID in range(n_blobs[section]):
    if (blobID % 1000 == 0):
        print('Section %d Finished percentage: %f'%(section, (float(blobID)*100 / n_blobs[section]) ))

    temp_curr_blob_props = {}
    for tempProp in prop:
        temp_curr_blob_props[tempProp] = blob_prop_dic[section][tempProp][blobID]
    if not blob_idx_selection[section][blobID]:
#         print('Blob %d eliminated'%blobID)
        continue

    tempB1_idx_loc = temp_curr_blob_props['centroid']
    if o_fix_scan_size:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max)
    else:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max)


    for tempSec in scan_section:
#         print('Blbo %d left. Start scanning'%blobID)
        if tempSec not in secList_in_BlobPropDic:
            continue

        # Find blobs at the nearby location in the scaned section
        tempBlobInside = fun_blobs_in_polygen(blob_prop_dic[tempSec]['centroid'],temp_next_sec_range,coor_order='cr')
        tempBlobOutside = fun_blobs_out_polygen(blob_prop_dic[tempSec]['centroid'], temp_next_sec_range, coor_order='cr', margin=scan_parameters['scan_outside_margin'])
        tempBlobInsideIndex = np.where(tempBlobInside)[0]
        temp_num_blob = len(tempBlobInsideIndex)
        tempBlobOutsideIndex = np.where(tempBlobOutside)[0]
        temp_num_outblob = len(tempBlobOutsideIndex)

        ### Typical blobs###

        if temp_num_blob:
            temp_sim = {}
            for temp_prop in prop_for_comparison:
                      temp_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobInside],distance_type=temp_prop))
            temp_sim_matrix = np.column_stack((temp_sim[temp_prop] for temp_prop in prop_for_comparison))

            #### Blob comparison ####
            temp_weighted_sim = np.dot(temp_sim_matrix,compare_weight)
            temp_compare_result = temp_weighted_sim > o_simil_threshold
            if any(temp_compare_result.tolist()):
                typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                matched_paris.append([section,blobID,
                                      im_blob_prop[section][blobID],
                                      tempSec,
                                      tempBlobInsideIndex[temp_compare_result],
                                      im_blob_prop[tempSec][tempBlobInsideIndex[temp_compare_result]],
                                      temp_sim_matrix[temp_compare_result,:]])
        else:
            pass

        ### False-typical blobs###
        if temp_num_outblob:
            temp_false_sim = {}
            for temp_prop in prop_for_comparison:
                      temp_false_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobOutside],distance_type=temp_prop))
            temp_false_sim_matrix = np.column_stack((temp_false_sim[temp_prop] for temp_prop in prop_for_comparison))

            #### Blob comparison ####
            temp_weighted_false_sim = np.dot(temp_false_sim_matrix,compare_weight)
            temp_false_compare_result = temp_weighted_false_sim > o_simil_threshold
            if any(temp_false_compare_result.tolist()):
                false_typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                false_matched_paris.append([section,blobID,im_blob_prop[section][blobID],
                                            tempSec, 
                                            tempBlobOutsideIndex[temp_false_compare_result], 
                                            im_blob_prop[tempSec][tempBlobOutsideIndex[temp_false_compare_result]],
                                            temp_false_sim_matrix[temp_false_compare_result,:]])
        else:
            pass
fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='typical')
fun_save_matched_blobs_info(matched_paris=matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='matched')
fun_save_typical_blobs_info(typical_blobs=false_typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='false_typical')
fun_save_matched_blobs_info(matched_paris=false_matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='false_matched')

# data_typical_blobs[section] = typical_blobs
# data_matched_paris[section] = matched_paris
# data_false_typical_blobs[section] = false_typical_blobs
# data_false_matched_pairs[section] = false_matched_paris   

Section 154 Finished percentage: 0.000000
Section 154 Finished percentage: 89.206066
